# FLSim Tutorial: Sentiment Classification with LEAF's Sent140



## Introduction

In this tutorial, we will train a binary sentiment classifier on LEAF's Sent140 dataset with federated learning using FLSim. 


### Prerequisites

To get the most of this tutorial, you should be comfortable training machine learning models with **PyTorch** and familiar with the concept of **federated learning (FL)**. If you are unfamimiliar with either of them or could use a refresher, please take a look at the following resources before proceeding with the tutorial:

- McMahan & Ramage (2017): [Federated Learning: Collaborative Machine Learning without Centralized Training Data](https://ai.googleblog.com/2017/04/federated-learning-collaborative.html). A short blog post from Google AI introducing the main idea of FL in a beginner-friendly way.
- McMahan et al. (2017): [Communication-Efficient Learning of Deep Networks from Decentralized Data](https://arxiv.org/pdf/1602.05629.pdf). This paper first proposes the approach of federated learning. The described algorithm is now known as federated averaging (or FedAvg for short).
- PyTorch has [extensive tutorials](https://pytorch.org/tutorials/) on their website.
- If you're new to **sentiment classification**, you can find Pang and Lee's survey on the topic [here](https://www.cs.cornell.edu/home/llee/omsa/omsa-published.pdf). 

Now that you're familiar with PyTorch and FL and have a sense of sentiment classification, let's move on!

### Objectives

In this tutorial, you will learn how to 

1. Build a data pipeline for federated learning with FLSim,
2. Create a sentiment classification model compatible with FL training,
3. Set hyperparameters for FL training, and
4. Launch an FL training flow using FLSim.

## Training a sentiment classifier with FLSim

### 0. About the dataset

For this tutorial, we're using [LEAF's](https://leaf.cmu.edu/) [Sentiment140 (Sent140) dataset](https://leaf.cmu.edu/build/html/tutorials/sent140-md.html), which consists of 1.6 million tweets by 660k users. Note that the mean number of samples per user is 2.42 and the standard deviation is 4.71.

![Sent140 distribution of samples across users](https://leaf.cmu.edu/webpage/images/twitter_hist.png)

Before the next step in this tutorial, you need to download the dataset and partition the data by users. 
We've included a script, `get_data.sh`, which will download and preproces the data for you. 
In particular, we sample 1% of the entire dataset in a non-IID manner and
partition 90% of sampled users into train and 10% of sampled users into test (as opposed to individual samples).
We require all users to have at least one sample.

For more information on the various preprocessing options, see [here](https://github.com/TalwalkarLab/leaf/tree/master/data/sent140). You can find the LEAF paper [here](https://arxiv.org/pdf/1812.01097.pdf).


In [1]:
%cd ~/local
!sh FLSim/tutorials/get_data.sh

/data/users/jessicazhao
fatal: destination path 'leaf' already exists and is not an empty directory.


------------------------------
calculating JSON file checksums


checksums written to meta/dir-checksum.md5
Data for one of the specified preprocessing tasks has already been
generated. If you would like to re-generate data for this directory,
please delete the existing one. Otherwise, please remove the
respective tag(s) from the preprocessing command.


We can find the preprocessed training and test data here:

In [2]:
!ls leaf/data/sent140/data/train; ls leaf/data/sent140/data/test

all_data_0_01_keep_1_train_9.json
all_data_0_01_keep_1_test_9.json


Note: if you use different preprocessing options, you will need to change these!

In [3]:
TRAIN_DATA = "leaf/data/sent140/data/train/all_data_0_01_keep_1_train_9.json"
TEST_DATA = "leaf/data/sent140/data/test/all_data_0_01_keep_1_test_9.json"

We can now get a rough idea of the structure of the training data:

In [4]:
import json


with open(TRAIN_DATA, "r") as f:
    training_data = json.load(f)

    # get overall structure of the data
    for key, val in training_data.items():
        print(key, type(val), len(val))


users <class 'list'> 6008
num_samples <class 'list'> 6008
user_data <class 'dict'> 6008


We can compute the minimum, maximum, and mean number of samples per user:

In [5]:
print(f"Min # samples per user: {min(training_data['num_samples'])}")
print(f"Max # samples per user: {max(training_data['num_samples'])}")
print(
    f"Mean # samples per user: {round(sum(training_data['num_samples'])/len(training_data['num_samples']), 2)}"
)

Min # samples per user: 1
Max # samples per user: 87
Mean # samples per user: 2.42


Let us also look at the data for an example user:

In [6]:
EXAMPLE_USER = training_data["users"][0]
training_data["user_data"][EXAMPLE_USER]

{'x': [['1882285552',
   'Fri May 22 06:35:30 PDT 2009',
   'NO_QUERY',
   'abbyyyoung',
   "@michaelaline I want a pair real bad too!  TOM'S are awesome.",
   'training']],
 'y': [1]}

### 1. Data pipeline

Now, let us define how to build the data pipeline for federated learning:

1. To load the training and test data, we define a new dataset class, `Sent140Dataset`, which converts each user's tweets (features) into a `torch.Tensor`, discarding tweet metadata such as time, and stores each tweet's sentiment (label) as well.



In [7]:
import itertools
import re
import string
import unicodedata

import torch
from torch.utils.data import Dataset


# 1. Sent140Dataset will store the tweets and corresponding sentiment for each user.


class Sent140Dataset(Dataset):
    def __init__(self, data_root, max_seq_len):
        self.data_root = data_root
        self.max_seq_len = max_seq_len
        self.all_letters = {c: i for i, c in enumerate(string.printable)}
        self.num_letters = len(self.all_letters)
        self.UNK = self.num_letters

        with open(data_root, "r+") as f:
            self.dataset = json.load(f)

        self.data = {}
        self.targets = {}

        self.num_classes = 2  # binary sentiment classification

        # Populate self.data and self.targets
        for user_id, user_data in self.dataset["user_data"].items():
            self.data[user_id] = self.process_x(list(user_data["x"]))
            self.targets[user_id] = self.process_y(list(user_data["y"]))

    def __len__(self):
        return len(self.data)

    def __iter__(self):
        for user_id in self.data.keys():
            yield self.__getitem__(user_id)

    def __getitem__(self, user_id: str):
        if user_id not in self.data or user_id not in self.targets:
            raise IndexError(f"User {user_id} is not in dataset")

        return self.data[user_id], self.targets[user_id]

    def unicodeToAscii(self, s):
        return "".join(
            c
            for c in unicodedata.normalize("NFD", s)
            if unicodedata.category(c) != "Mn" and c in self.all_letters
        )

    def line_to_indices(self, line: str, max_seq_len: int):
        line_list = self.split_line(line)  # split phrase in words
        line_list = line_list
        chars = self.flatten_list([list(word) for word in line_list])
        indices = [
            self.all_letters.get(letter, self.UNK)
            for i, letter in enumerate(chars)
            if i < max_seq_len
        ]
        # Add padding
        indices = indices + [self.UNK] * (max_seq_len - len(indices))
        return indices

    def process_x(self, raw_x_batch):
        x_batch = [e[4] for e in raw_x_batch]  # e[4] contains the actual tweet
        x_batch = [self.line_to_indices(e, self.max_seq_len) for e in x_batch]
        x_batch = torch.LongTensor(x_batch)
        return x_batch

    def process_y(self, raw_y_batch):
        y_batch = [int(e) for e in raw_y_batch]
        return y_batch

    def split_line(self, line):
        """
        Split given line/phrase into list of words

        Args:
            line: string representing phrase to be split

        Return:
            list of strings, with each string representing a word
        """
        return re.findall(r"[\w']+|[.,!?;]", line)

    def flatten_list(self, nested_list):
        return list(itertools.chain.from_iterable(nested_list))


2. We can now load the train and test dataset.


In [8]:
MAX_SEQ_LEN = 25


# 2. Load the train and test datasets.
train_dataset = Sent140Dataset(
    data_root=TRAIN_DATA,
    max_seq_len=MAX_SEQ_LEN,
)
test_dataset = Sent140Dataset(
    data_root=TEST_DATA,
    max_seq_len=MAX_SEQ_LEN,
)


Recall our `EXAMPLE_USER` from earlier? Their data now looks like this:

In [9]:
train_dataset[EXAMPLE_USER]

(tensor([[22, 18, 12, 17, 10, 14, 21, 10, 21, 18, 23, 14, 44, 32, 10, 23, 29, 10,
          25, 10, 18, 27, 27, 14, 10]]),
 [1])

To complete our data pipeline, we only need to

3. Create a data loader, which will batchify training, eval, and test data. There is no need to create a sharder since the data is already sharded. For each dataset, the data loader splits each client's data into batches of size `batch_size`. We choose not to drop the last batch.

4. Lastly, wrap the data loader with a data provider and return it. 
The data provider creates clients from the groupings in the data loader and adds metadata (e.g. number of examples, number of batches per client). 
Our data is now formatted such that the trainer will accept it.

In [10]:
from flsim.baselines.data_providers import LEAFDataLoader, LEAFDataProvider


# 3. Batchify training, eval, and test data. Note that train_dataset is already sharded.
dataloader = LEAFDataLoader(
    train_dataset,
    test_dataset,
    test_dataset,
    batch_size=32,
    drop_last=False,
)

# 4. Wrap the data loader with a data provider.
data_provider = LEAFDataProvider(dataloader)


Creating FL User: 6008user [00:00, 9331.22user/s]
Creating FL User: 668user [00:00, 9436.61user/s]
Creating FL User: 668user [00:00, 9783.38user/s]


### 2. Create the model

Now, let's see how we can create a model that is compatible with FL-training.

1. First, we define a standard, non-FL sentiment classification pytorch `nn.Module`; in this tutorial we use a simple char-LSTM.

In [11]:
from torch import nn


# 1. Define our model, a simple char-LSTM.

class CharLSTM(nn.Module):
    def __init__(
        self,
        num_classes,
        n_hidden,
        num_embeddings,
        embedding_dim,
        max_seq_len,
        dropout_rate,
    ):
        super().__init__()
        self.dropout_rate = dropout_rate
        self.n_hidden = n_hidden
        self.num_classes = num_classes
        self.max_seq_len = max_seq_len
        self.num_embeddings = num_embeddings

        self.embedding = nn.Embedding(
            num_embeddings=self.num_embeddings, embedding_dim=embedding_dim
        )
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=self.n_hidden,
            num_layers=2,
            batch_first=True,
            dropout=self.dropout_rate,
        )
        self.fc = nn.Linear(self.n_hidden, self.num_classes)
        self.dropout = nn.Dropout(p=self.dropout_rate)

    def forward(self, x):
        seq_lens = torch.sum(x != (self.num_embeddings - 1), 1) - 1
        x = self.embedding(x)  # [B, S] -> [B, S, E]
        out, _ = self.lstm(x)  # [B, S, E] -> [B, S, H]
        out = out[torch.arange(out.size(0)), seq_lens]
        out = self.fc(self.dropout(out))  # [B, S, H] -> # [B, S, C]
        return out


We initialize our model wich such parameters that it is compatible with our dataset.

In [12]:
model = CharLSTM(
    num_classes=train_dataset.num_classes,
    n_hidden=100,
    num_embeddings=train_dataset.num_letters + 1,
    embedding_dim=100,
    max_seq_len=MAX_SEQ_LEN,
    dropout_rate=0.1,
)

model


CharLSTM(
  (embedding): Embedding(101, 100)
  (lstm): LSTM(100, 100, num_layers=2, batch_first=True, dropout=0.1)
  (fc): Linear(in_features=100, out_features=2, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

After we have our standard PyTorch model, we can

2. Create a `torch.device` and choose where the model will be allocated (CUDA or CPU). 

3. Wrap the pytorch module with the FLSim `FLModel`. `FLModel` is accepted by the trainer and handles moving our model, data, and predictions to GPU if desired. It also collects and returns metrics for each batch it predicts on. You can find its implementation [here](https://github.com/facebookresearch/FLSim/blob/main/baselines/models/cv_model.py)

4. Move the model to GPU and enable CUDA if desired.

The model now supports FL training!

In [13]:
import torch
from flsim.baselines.models.cv_model import FLModel


USE_CUDA = True

# 2. Choose where the model will be allocated.
cuda_enabled = torch.cuda.is_available() and USE_CUDA
device = torch.device(f"cuda:{0}" if cuda_enabled else "cpu")

# 3. Wrap the model in FLModel.
global_model = FLModel(model, device)

# 4. Enable CUDA if desired.
if cuda_enabled:
    global_model.fl_cuda()


### 3. Hyperparameters

We can represent the hyperparameters for FL training in a JSON config.

This config is passed to the FL trainer.

In [14]:
json_config = {
    "trainer": {
        "_base_": "base_sync_trainer",
        # there are different types of aggegator
        # fed avg with lr requires a learning rate, wheras e.g. fed_avg doesn't
        "aggregator": {
            "_base_": "base_fed_avg_with_lr_sync_aggregator",
            # server's learning rate
            "lr": 0.7,
            # server's global momentum
            "momentum": 1,
            # reduce client models into a single model by taking their weighted sum
            "reducer": {"_base_": "base_reducer", "reduction_type": "WEIGHTED_SUM"},
        },
        "client": {
            # number of client's local epochs
            "epochs": 1,
            "optimizer": {
                "_base_": "base_optimizer_sgd",
                # client's local learning rate
                "lr": 1,
                # client's local momentum
                "momentum": 0,
            },
            "lr_scheduler": {
                # normalize the learning rate by the number of examples in the batch
                "_base_": "base_lr_batch_size_normalizer_scheduler",
                "local_lr_normalizer": 32,
            },
        },
        # type of user selection sampling
        "active_user_selector": {
            "_base_": "base_uniformly_random_active_user_selector"
        },
        # number of users per round for aggregation
        "users_per_round": 10,
        # total number of global epochs
        # total #rounds = ceil(total_users / users_per_round) * epochs
        "epochs": 1,
        # frequency of reporting train metrics
        "train_metrics_reported_per_epoch": 4,
        # keep the trained model always (as apposed to only when it
        # performs better than the previous model on eval)
        "always_keep_trained_model": False,
        # frequency of evaluation per epoch
        "eval_epoch_frequency": 1,
        "do_eval": True,
        # should we report train metrics after global aggregation
        "report_train_metrics_after_aggregation": True,
    }
}


Even though we recommend a JSON config for ease of representation, FLSim is compatible with the Hydra config system and can work with YAML configs just like any other [PyTorch Lightning](https://www.pytorchlightning.ai/) project. Here, we convert the JSON config to OmegaConf via Hydra for consumption by FLSim. 

In [15]:
import flsim.configs
from flsim.utils.config_utils import fl_config_from_json
from omegaconf import OmegaConf


cfg = fl_config_from_json(json_config)
print(OmegaConf.to_yaml(cfg))


trainer:
  _target_: flsim.trainers.sync_trainer.SyncTrainer
  _recursive_: false
  epochs: 1.0
  do_eval: true
  always_keep_trained_model: false
  timeout_simulator:
    _target_: ???
    _recursive_: false
  train_metrics_reported_per_epoch: 4
  eval_epoch_frequency: 1.0
  active_user_selector:
    _target_: flsim.active_user_selectors.simple_user_selector.UniformlyRandomActiveUserSelector
    _recursive_: false
    user_selector_seed: null
    random_with_replacement: false
  report_train_metrics: true
  report_train_metrics_after_aggregation: true
  use_train_clients_for_aggregation_metrics: true
  client:
    _target_: flsim.clients.base_client.Client
    _recursive_: false
    epochs: 1
    optimizer:
      _target_: flsim.optimizers.local_optimizers.LocalOptimizerSGD
      _recursive_: false
      lr: 1.0
      momentum: 0.0
      weight_decay: 0.0
    lr_scheduler:
      _target_: flsim.optimizers.optimizer_scheduler.LRBatchSizeNormalizer
      _recursive_: false
      base_lr

### 4. Training
Recall that we already built the data provider and created a model compatible with FL training. 
Now, to launch the FL training flow we only need to take a few more steps:

1. First, we need to create a metric reporter, which will collect, evaluate, and report relevent training, aggretaion, and evaluation/test metrics.
You can find its implementation [here](https://github.com/facebookresearch/FLSim/blob/main/tutorials/metrics_reporter/fl_metrics_reporter.py).

2. We also need to instantiate the trainer with the model and hyperparameter config we defined earlier.

In [16]:
from flsim.interfaces.metrics_reporter import Channel
from flsim.tutorials.metrics_reporter.fl_metrics_reporter import MetricsReporter
from hydra.utils import instantiate


# 1. Create a metric reporter.
metrics_reporter = MetricsReporter([Channel.TENSORBOARD, Channel.STDOUT])


# 2. Instantiate the trainer.
trainer_config = cfg.trainer
trainer = instantiate(trainer_config, model=global_model, cuda_enabled=cuda_enabled)


Finally, we're ready to run FL training given the above JSON config. We can utilize `eval_score` to store the evaluation metrics.

In [17]:
# Launch FL training.
final_model, eval_score = trainer.train(
    data_provider=data_provider,
    metric_reporter=metrics_reporter,
    num_total_users=data_provider.num_users(),
    distributed_world_size=1,
)


Round:  25%|██▍       | 150/601 [07:46<41:05,  5.47s/round]

Train finished Global Round: 151
(epoch = 1, round = 151, global round = 151), Loss/Training: 3044.5953419751395


(epoch = 1, round = 151, global round = 151), Accuracy/Training: 49.58559588453844
(epoch = 1, round = 151, global round = 151), round_to_target/Training: 10000000000.0
reporting (epoch = 1, round = 151, global round = 151) for aggregation


Round:  25%|██▌       | 151/601 [07:51<40:24,  5.39s/round]

(epoch = 1, round = 151, global round = 151), Loss/Aggregation: 2793.5030517578125
(epoch = 1, round = 151, global round = 151), Accuracy/Aggregation: 59.09090909090909
(epoch = 1, round = 151, global round = 151), round_to_target/Aggregation: 10000000000.0


...

Round: 100%|█████████▉| 600/601 [18:45<00:01,  1.17s/round]

Train finished Global Round: 601
(epoch = 1, round = 601, global round = 601), Loss/Training: 23256.68788273116


(epoch = 1, round = 601, global round = 601), Accuracy/Training: 54.00398974066686
(epoch = 1, round = 601, global round = 601), round_to_target/Training: 10000000000.0
reporting (epoch = 1, round = 601, global round = 601) for aggregation
(epoch = 1, round = 601, global round = 601), Loss/Aggregation: 21179.43720703125
(epoch = 1, round = 601, global round = 601), Accuracy/Aggregation: 70.58823529411765
(epoch = 1, round = 601, global round = 601), round_to_target/Aggregation: 10000000000.0
Running (epoch = 1, round = 601, global round = 601) for Eval


(epoch = 1, round = 601, global round = 601), Loss/Eval: 24499.7828654215
(epoch = 1, round = 601, global round = 601), Accuracy/Eval: 55.10344827586207
(epoch = 1, round = 601, global round = 601), round_to_target/Eval: 10000000000.0


Epoch:   0%|          | 0/1 [18:58<?, ?epoch/s]


After training finishes, we evaluate the model and report the test set accuracy before concluding this tutorial.

In [18]:
# We can now test our model.
trainer.test(
    data_iter=data_provider.test_data(),
    metric_reporter=MetricsReporter([Channel.STDOUT]),
)


Running (epoch = 1, round = 1, global round = 1) for Test


(epoch = 1, round = 1, global round = 1), Loss/Test: 24499.782963370133
(epoch = 1, round = 1, global round = 1), Accuracy/Test: 55.10344827586207
(epoch = 1, round = 1, global round = 1), round_to_target/Test: 10000000000.0


{'Accuracy': 55.10344827586207, 'round_to_target': 10000000000.0}

## Summary

In this tutorial, we first showed how to get and preprocess LEAF's Sent140 dataset. 
We then built a data provider by splitting each user's data into batches. 
We defined a simple char-LSTM as our model, wrapped it with a model compatible with FL training, and moved it to GPU. 
Lastly, we set the hyperparameters for FL training, launched the training flow, and evaluated our model.

### Additional resources
- [FLSim tutorials](https://github.com/facebookresearch/FLSim/tree/main/tutorials) - check out our other tutorial on sentiment classification.
- Kairouz et al. (2021): [Advances and Open Problems in Federated Learning](https://arxiv.org/pdf/1912.04977.pdf). As the title suggests, an in-depth overview of advances and open problems in FL.
- If you're interested in federated learning with **differential privacy**, take a look at [Opacus](https://opacus.ai/), a library that enables training PyTorch models with differential privacy. 
You can find a blog post introducing Opacus [here](https://ai.facebook.com/blog/introducing-opacus-a-high-speed-library-for-training-pytorch-models-with-differential-privacy/).

